# 2: Extracting Global Feature Weighting:

### Pedagogical 
1. Perturbation Method 
2. Sensitivity (Im et al. 2007) 

### Decompositional
1. Garson's Algorithm. 
2. Connection Weights. 

In [1]:
import pandas as pd
import numpy as np
import pickle

from keras.models import load_model

from copy import deepcopy

from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error

Using TensorFlow backend.


## Load Models and Data:

In [2]:
df = pd.read_csv("processed_df.csv")

In [3]:
knn_clf = pickle.load(open("k-nn_model.sav", 'rb'))
model = load_model("NN.h5")

In [4]:
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

In [5]:
# To keep track of what the numpy matrix columns are
feature_names = df.columns

## Pedagogical
### Sensitivity (Im et al. 2007)

$$
S_i = \frac{(\sum_{L} \frac{|P^0 - P^i|}{P^0})}{n}
$$

Where $P^0$ is the normal prediction value for each training instance after training. <br>
$P^i$ is the modified prediction when input node is removed. <br>
$L$ is the set of training data. <br>
$n$ is the number of training data instances.

In [6]:
# Iterate though each feature, remove it from the input data and evalutate above forumla for get weights

sensitivity_weights = dict()

for i in range(len(feature_names)):
    feature = feature_names[i]
    temp_df = deepcopy(X_test)
    
    # Remove Input
    temp_df = temp_df.T
    temp_df[i] = 0
    temp_df = temp_df.T
    
    # Get accuracy
    numerator = 0
    denomonator = len(X_test)
    
    for j in range(len(X_test)):
        
        # Get index of actual class predicted
        idx = model.predict_classes(np.array([X_test[j]]))[0]
        p0 = model.predict_proba(np.array([X_test[j]]))[0][idx]
        pi = model.predict_proba(np.array([temp_df[j]]))[0][idx]
        numerator += (abs(p0 - pi) / p0)
        
    si = numerator / denomonator
    sensitivity_weights[feature] = si
    
sensitivity_weights

{'b_b': 0.026597054205614007,
 'b_o': 0.10770873571663085,
 'b_x': 0.1002009721243068,
 'b.1_b': 0.021807881862042137,
 'b.1_o': 0.08304163931169267,
 'b.1_x': 0.07676830626061491,
 'b.2_b': 0.038899741471148955,
 'b.2_o': 0.07064695273250411,
 'b.2_x': 0.07287364337547016,
 'b.3_b': 0.021886903727944745,
 'b.3_o': 0.04269175323285053,
 'b.3_x': 0.046830474603358305,
 'b.4_b': 0.0208708677208937,
 'b.4_o': 0.07805900812185343,
 'b.4_x': 0.08999469549035101,
 'b.5_b': 0.020111748482276752,
 'b.5_o': 0.09119032952898769,
 'b.5_x': 0.09548258012917421,
 'b.6_b': 0.048184033657458505,
 'b.6_o': 0.10313898477006088,
 'b.6_x': 0.09567416869411562,
 'b.7_b': 0.030126868445686825,
 'b.7_o': 0.056248243340339246,
 'b.7_x': 0.061666405396998976,
 'b.8_b': 0.03526915249656314,
 'b.8_o': 0.060737560104274495,
 'b.8_x': 0.06145761091289497,
 'b.9_b': 0.023055169739953363,
 'b.9_o': 0.050731543751415774,
 'b.9_x': 0.07752829029375098,
 'b.10_b': 0.017968783557275204,
 'b.10_o': 0.05083243805046371,


In [7]:
np.save('sensitivity_weights.npy', sensitivity_weights) 

### Perturbation Method

$ x = x +- \sigma $

Where $\sigma$ is a perturbation of 0-20% of the input $x$. There was a recent paper by Runbo et al. http://www.jcomputers.us/vol6/jcp0607-16.pdf showing the optimal perturbation range to be 20%.

Theodor et al. https://www.dcl.hpi.uni-potsdam.de/papers/papers/heinze-feature-salience.pdf also showed perturbation to be the most promising method of retrieving feature ranking from a NN when compared to their own algorithm and connection weights.

##### Our Proposed Weighting Formulation
$ W_i = \frac{\sum_{j=1}^{n}\delta (\sigma, L_j)}{2n} $ 

### First +20%

In [8]:
perturb_up = dict()
perturbation_range = 0.4  # +20% Since data is normalised between -1 => +1

# Prepare dicitonary for feature weights
for feature in feature_names:
    perturb_up[feature] = 0

# Get original RMSE before perturbation
for i in range(len(X_test)):
    
    # Which index is the predicted class?
    idx = model.predict_classes(np.array([X_test[i]]))[0]
    before_perturb = model.predict_proba(np.array([X_test[i]]))[0][idx]

    for j in range(len(feature_names)):
        perturb_instance = deepcopy(X_test[i])
        perturb_instance[j] += perturbation_range
        after_perturb = model.predict_proba(np.array([perturb_instance]))[0][idx]
        change = abs(before_perturb - after_perturb)
        perturb_up[feature_names[j]] += change
        
perturb_up

{'b_b': 46.32006114721298,
 'b_o': 188.9316775649786,
 'b_x': 180.41409875452518,
 'b.1_b': 38.14100280404091,
 'b.1_o': 143.88720712065697,
 'b.1_x': 136.5660767108202,
 'b.2_b': 67.09883067011833,
 'b.2_o': 123.51666498184204,
 'b.2_x': 128.18925294280052,
 'b.3_b': 38.83304759860039,
 'b.3_o': 76.06753608584404,
 'b.3_x': 82.23695069551468,
 'b.4_b': 36.72442018985748,
 'b.4_o': 139.14523144066334,
 'b.4_x': 157.05636577308178,
 'b.5_b': 34.6191321015358,
 'b.5_o': 161.9488182067871,
 'b.5_x': 166.8279329240322,
 'b.6_b': 87.05623027682304,
 'b.6_o': 183.36360585689545,
 'b.6_x': 166.03831535577774,
 'b.7_b': 53.135736018419266,
 'b.7_o': 102.9605841934681,
 'b.7_x': 108.83180756866932,
 'b.8_b': 61.618211179971695,
 'b.8_o': 108.08173394203186,
 'b.8_x': 106.51241202652454,
 'b.9_b': 40.367498099803925,
 'b.9_o': 89.49234491586685,
 'b.9_x': 132.562737762928,
 'b.10_b': 32.08538341522217,
 'b.10_o': 90.27656599879265,
 'b.10_x': 81.81504538655281,
 'b.11_b': 37.8076776266098,
 'b.1

### Now -20%

In [9]:
perturb_down = dict()
perturbation_range = -0.4  # +20% Since data is normalised between -1 => +1

# Prepare dicitonary for feature weights
for feature in feature_names:
    perturb_down[feature] = 0

# Get original RMSE before perturbation
for i in range(len(X_test)):
    
    # Which index is the predicted class?
    idx = model.predict_classes(np.array([X_test[i]]))[0]
    before_perturb = model.predict_proba(np.array([X_test[i]]))[0][idx]

    for j in range(len(feature_names)):
        perturb_instance = deepcopy(X_test[i])
        perturb_instance[j] += perturbation_range
        after_perturb = model.predict_proba(np.array([perturb_instance]))[0][idx]
        change = abs(before_perturb - after_perturb)
        perturb_down[feature_names[j]] += change
        
perturb_down

{'b_b': 46.06359213590622,
 'b_o': 191.2170085310936,
 'b_x': 178.8202322125435,
 'b.1_b': 37.92730578780174,
 'b.1_o': 144.82646584510803,
 'b.1_x': 136.8271549642086,
 'b.2_b': 67.80526196956635,
 'b.2_o': 124.9601993560791,
 'b.2_x': 124.37712714076042,
 'b.3_b': 38.728769689798355,
 'b.3_o': 77.43737417459488,
 'b.3_x': 79.93283864855766,
 'b.4_b': 36.7642747759819,
 'b.4_o': 143.3202024102211,
 'b.4_x': 151.26348100602627,
 'b.5_b': 34.89969062805176,
 'b.5_o': 165.25655284523964,
 'b.5_x': 158.74074086546898,
 'b.6_b': 86.64014154672623,
 'b.6_o': 180.64388439059258,
 'b.6_x': 166.58650417625904,
 'b.7_b': 53.65358677506447,
 'b.7_o': 105.55892702937126,
 'b.7_x': 104.53268027305603,
 'b.8_b': 61.93948137760162,
 'b.8_o': 108.15967224538326,
 'b.8_x': 105.21436887979507,
 'b.9_b': 40.302173525094986,
 'b.9_o': 89.9950715303421,
 'b.9_x': 131.56221842765808,
 'b.10_b': 31.87360328435898,
 'b.10_o': 93.14361724257469,
 'b.10_x': 78.65332755446434,
 'b.11_b': 38.081851094961166,
 'b

In [10]:
# Average Scores going up and down
perturb_weights = dict()

for key, value in perturb_up.items():
    # Divide the values by the number of training samples to normalise the results
    # Also divide by two to normalise the up and down sampling
    perturb_weights[key] = ( perturb_up[key] + perturb_down[key] ) / ( 2 * len(X_train) )
    
perturb_weights

{'b_b': 0.000759733990815125,
 'b_o': 0.0031262227475005937,
 'b_x': 0.0029542296954528672,
 'b.1_b': 0.0006255617482881797,
 'b.1_o': 0.00237429007373162,
 'b.1_x': 0.0022482996025906974,
 'b.2_b': 0.0011094086565763541,
 'b.2_o': 0.0020433952659368515,
 'b.2_x': 0.0020770261520029684,
 'b.3_b': 0.0006378438921743317,
 'b.3_o': 0.0012623759067470306,
 'b.3_x': 0.0013336331360532265,
 'b.4_b': 0.0006043478204427581,
 'b.4_o': 0.002322906528378984,
 'b.4_x': 0.0025355250557492436,
 'b.5_b': 0.0005717008448156871,
 'b.5_o': 0.0026908336435199566,
 'b.5_x': 0.002677373962084714,
 'b.6_b': 0.0014284241103910303,
 'b.6_o': 0.0029934826500615792,
 'b.6_x': 0.0027354014764147763,
 'b.7_b': 0.0008782016677095701,
 'b.7_o': 0.001714798612029929,
 'b.7_x': 0.0017546421697510308,
 'b.8_b': 0.0010160994453747806,
 'b.8_o': 0.0017783010377254533,
 'b.8_x': 0.0017411741850848652,
 'b.9_b': 0.000663401904810024,
 'b.9_o': 0.00147604783261685,
 'b.9_x': 0.002172080231830478,
 'b.10_b': 0.0005259785090

In [11]:
np.save('perturb_weights.npy', perturb_weights) 

## Decompositional

### Garson's Algorithm
Shown by Olden et al. 2002 ftp://gis.msl.mt.gov/Maxell/Models/Predictive_Modeling_for_DSS_Lincoln_NE_121510/Modeling_Literature/Olden_ANN's.pdf

#### Step 1: Calculate the product of each input weight with the hidden layer weight

In [12]:
first_layer_weights = model.layers[0].get_weights()[0]
second_layer_weights = model.layers[2].get_weights()[0]

In [13]:
# (hidden_nodes, inputs)
W1 = first_layer_weights.T
W2 = second_layer_weights.T
Qih_dict = dict()

for h in range(len(W1)):
    for i in range(len(W1[0])):
        Qih_dict["H_" + str(h) + "I_" + str(i)] = W1[h][i] * W2[0][h]

#### Step 2: Relative Contribution
For each input neuron to the outgoing signal of each hidden neuron.

In [14]:
Qih_dict2 = dict()

for h in range(len(W1)):
    for i in range(len(W1[0])):
        
        numerator = abs(Qih_dict["H_" + str(h) + "I_" + str(i)])
        denomonator = 0
        
        for j in range(len(W1[0])):
            denomonator += abs(Qih_dict["H_" + str(h) + "I_" + str(j)])
            
        Qih_dict2["H_" + str(h) + "I_" + str(i)] = numerator / denomonator

In [15]:
# Also sum input neuron contributions
for i in range(len(W1[0])):
    total = 0
    for h in range(len(W1)):
        total += Qih_dict2["H_" + str(h) + "I_" + str(i)]
    Qih_dict2["Sum_input_" + str(i)] = total

#### Step 3: Relative Contribution of each input variable

In [16]:
garsons_weights = dict()

for i in range(len(W1[0])):
    total = 0
    for j in range(len(W1[0])):
        total += Qih_dict2["Sum_input_" + str(j)]
    garsons_weights["Input_" + str(i)] = Qih_dict2["Sum_input_" + str(i)] / total

In [17]:
for i in range(len(feature_names)):
    feature = feature_names[i]
    garsons_weights[feature] = garsons_weights.pop("Input_" + str(i)) 

In [18]:
garsons_weights

{'b_b': 0.0073632578410361,
 'b_o': 0.009568475805248838,
 'b_x': 0.009645355370073078,
 'b.1_b': 0.00677980615420967,
 'b.1_o': 0.007645554340371482,
 'b.1_x': 0.008920309466735991,
 'b.2_b': 0.007438713590688926,
 'b.2_o': 0.008437339756292692,
 'b.2_x': 0.00791258906190479,
 'b.3_b': 0.006290769234965503,
 'b.3_o': 0.006694756258750052,
 'b.3_x': 0.006468724719843546,
 'b.4_b': 0.006280723132315391,
 'b.4_o': 0.007265667669683982,
 'b.4_x': 0.007049859686959626,
 'b.5_b': 0.005820874125947884,
 'b.5_o': 0.007416995753960971,
 'b.5_x': 0.006750409073813505,
 'b.6_b': 0.009029141171339274,
 'b.6_o': 0.012466281257252614,
 'b.6_x': 0.011231167601045512,
 'b.7_b': 0.007567591658366181,
 'b.7_o': 0.010600622332310322,
 'b.7_x': 0.01037749028388634,
 'b.8_b': 0.008006417324752973,
 'b.8_o': 0.009125702326479945,
 'b.8_x': 0.008436244951569355,
 'b.9_b': 0.006149973405336414,
 'b.9_o': 0.00741749994607946,
 'b.9_x': 0.007535720394419056,
 'b.10_b': 0.005818641716626483,
 'b.10_o': 0.006681

In [19]:
np.save('garsons_weights.npy', garsons_weights) 

### Connection Weights Algorithm
Introducted by Olden et al. ftp://gis.msl.mt.gov/Maxell/Models/Predictive_Modeling_for_DSS_Lincoln_NE_121510/Modeling_Literature/Olden_ANN's.pdf

$ R_{ij} = \sum_{k=1}^{H}W_{ik}.W_{kj} $

$ R_{ij} = \sum_{k=1}^{H} \sum_{j=1}^{C} W_{ik}.W_{kj} $  -- Our Proposal

where $C$ is the number of softmax outputs

Where $k$ is a hidden neuron, $i$ is the input neuron and $j$ is the output neuron.

In [20]:
connection_weights = dict()
W1 = first_layer_weights
W2 = second_layer_weights

for i in range(len(W1)):
    total = 0
    for h in range(len(W1[i])):
        total += sum(W1[i][h] * W2[h])
    connection_weights["Input_" + str(i)] = total

In [21]:
W1.shape

(126, 126)

In [22]:
for i in range(len(feature_names)):
    feature = feature_names[i]
    connection_weights[feature] = connection_weights.pop("Input_" + str(i)) 

In [23]:
connection_weights

{'b_b': -0.6485695406299783,
 'b_o': -0.7277856014306963,
 'b_x': 0.5226681960430142,
 'b.1_b': 0.6674552244194274,
 'b.1_o': -0.12873803154980124,
 'b.1_x': 0.03883566349395551,
 'b.2_b': 0.26657011335328207,
 'b.2_o': -0.7290730002787313,
 'b.2_x': 0.30775065203852137,
 'b.3_b': 0.28655723187330295,
 'b.3_o': -1.2703318637620669,
 'b.3_x': 0.6484651035534625,
 'b.4_b': 0.15929409473210399,
 'b.4_o': -1.382568983452984,
 'b.4_x': 0.3001578920124075,
 'b.5_b': 0.9716607851714798,
 'b.5_o': -1.982741534258821,
 'b.5_x': 0.2676531325378164,
 'b.6_b': -0.8592042603431764,
 'b.6_o': 2.055073080227885,
 'b.6_x': -2.2161182520394505,
 'b.7_b': -0.3542895824466541,
 'b.7_o': -0.31606357059354195,
 'b.7_x': 0.41427646090915005,
 'b.8_b': -0.5536272074280078,
 'b.8_o': -0.3837147784252011,
 'b.8_x': -0.3262855570847023,
 'b.9_b': 0.4639579114937078,
 'b.9_o': -0.49040369260910666,
 'b.9_x': -0.5615325421376838,
 'b.10_b': 0.42240530856452096,
 'b.10_o': -1.0587100398533948,
 'b.10_x': 0.2339116

In [24]:
np.save('connection_weights.npy', connection_weights) 